In [21]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================


"""MNIST model training using TPUs.
This program demonstrates training of the convolutional neural network model
defined in mnist.py on Google Cloud TPUs (https://cloud.google.com/tpu/).
If you are not interested in TPUs, you should ignore this file.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import keras
from keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot as plt 
%matplotlib inline
import glob
import cv2
from collections import Counter
from IPython.core.display import display, HTML
import random
import numpy as np
import tensorflow as tf  # pylint: disable=g-bad-import-order

# For open source environment, add grandparent directory for import
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(sys.path[0]))))

# from official.mnist import dataset  # pylint: disable=wrong-import-position
# from official.mnist import mnist  # pylint: disable=wrong-import-position

In [4]:
# Cloud TPU Cluster Resolver flags
tf.flags.DEFINE_string(
    "tpu", default=None,
    help="The Cloud TPU to use for training. This should be either the name "
    "used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 "
    "url.")
tf.flags.DEFINE_string(
    "tpu_zone", default=None,
    help="[Optional] GCE zone where the Cloud TPU is located in. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")
tf.flags.DEFINE_string(
    "gcp_project", default=None,
    help="[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

# Model specific parameters
tf.flags.DEFINE_string("data_dir", "",
                       "Path to directory containing the Omniglot dataset")
tf.flags.DEFINE_string("model_dir", None, "Estimator model_dir")
tf.flags.DEFINE_integer("batch_size", 128,
                        "Mini-batch size for the training. Note that this "
                        "is the global batch size and not the per-shard batch.")
tf.flags.DEFINE_integer("train_steps", 200, "Total number of training steps.")
tf.flags.DEFINE_integer("eval_steps", 60,
                        "Total number of evaluation steps. If `0`, evaluation "
                        "after training is skipped.")

tf.flags.DEFINE_float("learning_rate", 3e-4, "Learning rate.")


tf.flags.DEFINE_bool("use_tpu", True, "Use TPUs rather than plain CPUs")
tf.flags.DEFINE_integer("iterations", 1000,
                        "Number of iterations per TPU training loop.")
tf.flags.DEFINE_integer("num_shards", 8, "Number of shards (TPU chips).")

FLAGS = tf.flags.FLAGS

In [27]:
def metric_fn(labels, logits):
    accuracy = tf.metrics.accuracy(
    labels=labels, predictions = logits)
    return {"accuracy": accuracy}

In [32]:
def model_fn(features, labels, mode, params):
    """model_fn constructs the ML model used to predict handwritten digits."""

    del params
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        raise RuntimeError("mode {} is not supported yet".format(mode))
    image1,image2  = features
    
    if isinstance(image, dict):
        image = features["image"]
        
    model = create_model()
    logits = model([image1,image2], training=(mode == tf.estimator.ModeKeys.TRAIN))
    loss = tf.losses.log_loss(labels=labels, predictions=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(
        FLAGS.learning_rate,
        tf.train.get_global_step(),
        decay_steps=100000,
        decay_rate=0.96)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        if FLAGS.use_tpu:
            optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
        return tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=optimizer.minimize(loss, tf.train.get_global_step()))

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, loss=loss, eval_metrics=(metric_fn, [labels, logits]))

In [29]:
train_data_gen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization = True,
    rotation_range =10,
    width_shift_range = 0.05,
    height_shift_range = 0.05,
    shear_range = 0.05,
    zoom_range = 0.05)
    #rescale = 1./255

In [30]:
def flip(p):
    # bigger p = less TRUE
    # True - in same class
    return random.random() < p
def batch_function(batch_size,features,lengthes,validation):
    batch_features = np.zeros((batch_size, 2,105, 105,1))
    batch_labels = np.zeros((batch_size,1))
    for i in range(batch_size):
        if flip(distribution):
            not_insame_class = True
            while not_insame_class:
                indecies= np.random.choice(len(features),2)
                not_insame_class = np.argmax(lengthes-indecies[0]>0) != np.argmax(lengthes-indecies[1]>0)
            batch_labels[i] = not_insame_class
        else:
                
            first = np.searchsorted(lengthes,np.random.randint(1,len(features)))
            indecies = np.zeros(2)
            while abs(indecies[0]-indecies[1]) < 20 : indecies = np.random.choice(range(lengthes[first-1],lengthes[first]),2)
            batch_labels[i] = True
    
        image1,image2 = features[indecies[0]].reshape(105,105,1),features[indecies[1]].reshape(105,105,1)
        if not validation and flip(0.5): 
            generator1 , generator2 = train_data_gen.flow(image1[np.newaxis,:],batch_size=1) , train_data_gen.flow(image2[np.newaxis,:],batch_size=1)
            image1,image2 = next(generator1),next(generator2)
        batch_features[i] = (image1,image2)
    return (batch_features,batch_labels)

In [18]:
def load_dataset(data_train_dir="images_background/", data_valid_dir = "images_evaluation/"):
    
    languages = [path.split("/")[-1] for path in glob.glob(dataset_train_path+"*")]
    languages_evaluation = [path.split("/")[-1] for path in glob.glob(dataset_evaluation_path+"*")]



    all_alphabets = {language:[cv2.cvtColor(cv2.imread(image),cv2.COLOR_RGB2GRAY) for image in glob.glob(dataset_train_path+language+"/*/*")]\
                                                                  for language in languages }
    all_images =  np.vstack([np.array(all_alphabets[key]) for key in all_alphabets])

    all_evaluation_alphabets = {language:[cv2.cvtColor(cv2.imread(image),cv2.COLOR_RGB2GRAY) for image in glob.glob(dataset_evaluation_path+language+"/*/*")]\
                                                                  for language in languages_evaluation }
    all_evaluation_images =  np.vstack([np.array(all_evaluation_alphabets[key]) for key in all_evaluation_alphabets])
    
    return all_alphabets,all_evaluation_alphabets,all_images,all_evaluation_images

In [8]:
# def eval_input_fn(params):
#     batch_size = params["batch_size"]
#     data_dir = params["data_dir"]
#     ds = dataset.test(data_dir).apply(
#       tf.contrib.data.batch_and_drop_remainder(batch_size))
#     images, labels = ds.make_one_shot_iterator().get_next()
#     return images, labels

In [38]:
def main(argv):
    del argv  # Unused.
    tf.logging.set_verbosity(tf.logging.INFO)

    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
      FLAGS.tpu,
      zone=FLAGS.tpu_zone,
      project=FLAGS.gcp_project
    )

    run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=FLAGS.model_dir,
      session_config=tf.ConfigProto(
          allow_soft_placement=True, log_device_placement=True),
      tpu_config=tf.contrib.tpu.TPUConfig(FLAGS.iterations, FLAGS.num_shards),
    )
    all_alphabets,all_evaluation_alphabets,all_images,all_evaluation_images = load_dataset()
    lengthes_train = np.insert(np.cumsum([len(all_alphabets[key]) for key in all_alphabets]),0,0)
    lengthes_val = np.insert(np.cumsum([len(all_evaluation_alphabets[key]) for key in all_evaluation_alphabets]),0,0)
    estimator = tf.contrib.tpu.TPUEstimator(
      model_fn=model_fn,
      use_tpu=FLAGS.use_tpu,
      train_batch_size=FLAGS.batch_size,
      eval_batch_size=FLAGS.batch_size,
      config=run_config)
    # TPUEstimator.train *requires* a max_steps argument.
    estimator.train(input_fn=batch_function(FLAGS.batch_size,all_images,lengthes_train,False), max_steps=FLAGS.train_steps)
    # TPUEstimator.evaluate *requires* a steps argument.
    # Note that the number of examples used during evaluation is
    # --eval_steps * --batch_size.
    # So if you change --batch_size then change --eval_steps too.
    if FLAGS.eval_steps:
        estimator.evaluate(input_fn=batch_function(FLAGS.batch_size,all_evaluation_images,lengthes_val,True), steps=FLAGS.eval_steps)

In [36]:
tf.contrib.tpu.TPUEstimator().train()

SyntaxError: invalid syntax (<ipython-input-36-f31e864442ea>, line 1)

In [12]:
def create_model():
    m = tf.keras.layers
    digit_input = tf.keras.Input(shape=(105,105,1))
    convolution = partial(m.Conv2D, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(0.005))

    x = convolution(64, (10, 10),input_shape=(105,105,1))(digit_input)
    x = m.MaxPooling2D()(x)
    x = convolution(128, (9, 9))(x)
    x = m.MaxPooling2D()(x)
    x = convolution(128, (7, 7))(x)
    x = m.MaxPooling2D()(x)
    x = convolution(256, (6, 6))(x)
    out = m.Flatten()(x)

    vision_model = tf.keras.models.Model(digit_input,out)

    letter1 = tf.keras.Input(shape=(105,105,1))
    letter2 = tf.keras.Input(shape=(105,105,1))

    out_1 = vision_model(letter1)
    out_2 = vision_model(letter2)

    def m_dist(A,B):
        return tf.keras.backend.sum(tf.keras.backend.abs(A-B),axis=1,keepdims=True)

    merged_vector = m.Lambda(lambda x:m_dist(x[0],x[1]), output_shape=lambda inp_shp:(inp_shp[0][0],1))([out_1,out_2])

    out_fin = m.Dense(4086,activation=tf.nn.sigmoid,kernel_regularizer=tf.keras.regularizers.l2(0.00015))(merged_vector)
    final_output = m.Dense(1,activation=tf.nn.sigmoid)(out_fin)

    final_model = tf.keras.models.Model(inputs=[letter1, letter2], outputs=final_output)
    return final_model

In [ ]:
if __name__ == "__main__":
    tf.app.run()